# Stance Detection for the Fake News Challenge

## Identifying Textual Relationships with Deep Neural Nets

### Check the problem context [here](https://drive.google.com/open?id=1KfWaZyQdGBw8AUTacJ2yY86Yxgw2Xwq0).

### Download files required for the project from [here](https://drive.google.com/open?id=10yf39ifEwVihw4xeJJR60oeFBY30Y5J8).

## Step1: Load the given dataset  

1. Mount the google drive

2. Import Glove embeddings

3. Import the test and train datasets

### Mount the google drive to access required project files

Run the below commands

In [0]:
from google.colab import drive

In [2]:
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


#### Path for Project files on google drive

**Note:** You need to change this path according where you have kept the files in google drive. 

In [0]:
project_path = "/content/drive/My Drive/DL Project/Sequence NLP/"

### Loading the Glove Embeddings

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'glove.6B.zip', 'r') as z:
  z.extractall()

# Load the dataset [5 Marks]

1. Using [read_csv()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) in pandas load the given train datasets files **`train_bodies.csv`** and **`train_stances.csv`**

2. Using [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) command in pandas merge the two datasets based on the Body ID. 

Note: Save the final merged dataset in a dataframe with name **`dataset`**.

In [0]:
import pandas as pd
import os
import numpy as np


In [0]:
os.chdir('/content/drive/My Drive/DL Project/Sequence NLP/')

In [0]:
train_bodies = pd.read_csv('train_bodies.csv')

In [0]:
train_stances = pd.read_csv('train_stances.csv')

In [15]:
train_bodies.columns

Index(['Body ID', 'articleBody'], dtype='object')

In [16]:
train_stances.columns

Index(['Headline', 'Body ID', 'Stance'], dtype='object')

In [0]:
train_df = train_bodies.merge(train_stances, on='Body ID')

In [18]:
train_df.head(10)

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated
5,0,A small meteorite crashed into a wooded area i...,Enormous 20-stone catfish caught with fishing ...,unrelated
6,0,A small meteorite crashed into a wooded area i...,Italian catches huge wels catfish; is it a rec...,unrelated
7,0,A small meteorite crashed into a wooded area i...,Not coming to a store near you: The pumpkin sp...,unrelated
8,0,A small meteorite crashed into a wooded area i...,One gunman killed in shooting on Parliament Hi...,unrelated
9,0,A small meteorite crashed into a wooded area i...,Soldier shot at war memorial in Canada,unrelated



<h2> Check1:</h2>
  
<h3> You should see the below output if you run `dataset.head()` command as given below </h3>

In [0]:
dataset.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated


## Step2: Data Pre-processing and setting some hyper parameters needed for model


#### Run the code given below to set the required parameters.

1. `MAX_SENTS` = Maximum no.of sentences to consider in an article.

2. `MAX_SENT_LENGTH` = Maximum no.of words to consider in a sentence.

3. `MAX_NB_WORDS` = Maximum no.of words in the total vocabualry.

4. `MAX_SENTS_HEADING` = Maximum no.of sentences to consider in a heading of an article.

In [0]:
MAX_NB_WORDS = 20000
MAX_SENTS = 20
MAX_SENTS_HEADING = 1
MAX_SENT_LENGTH = 20
VALIDATION_SPLIT = 0.2

### Download the `Punkt` from nltk using the commands given below. This is for sentence tokenization.

For more info on how to use it, read [this](https://stackoverflow.com/questions/35275001/use-of-punktsentencetokenizer-in-nltk).



In [20]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Tokenizing the text and loading the pre-trained Glove word embeddings for each token  [5 marks] 

Keras provides [Tokenizer API](https://keras.io/preprocessing/text/) for preparing text. Read it before going any further.

#### Import the Tokenizer from keras preprocessing text

In [0]:
from keras_preprocessing.text import Tokenizer

#### Initialize the Tokenizer class with maximum vocabulary count as `MAX_NB_WORDS` initialized at the start of step2. 

In [0]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)

#### Now, using fit_on_texts() from Tokenizer class, lets encode the data 

Note: We need to fit articleBody and Headline also to cover all the words.

In [0]:
tokenizer.fit_on_texts(train_df['articleBody']+train_df['Headline'])

In [27]:
tokenizer.word_counts

OrderedDict([('a', 478453),
             ('small', 5225),
             ('meteorite', 5773),
             ('crashed', 207),
             ('into', 25604),
             ('wooded', 554),
             ('area', 7132),
             ('in', 408065),
             ("nicaragua's", 526),
             ('capital', 6867),
             ('of', 441773),
             ('managua', 1823),
             ('overnight', 749),
             ('the', 1129038),
             ('government', 23631),
             ('said', 134456),
             ('sunday', 5416),
             ('residents', 3428),
             ('reported', 21926),
             ('hearing', 2514),
             ('mysterious', 853),
             ('boom', 1459),
             ('that', 234408),
             ('left', 11643),
             ('16', 2172),
             ('foot', 2441),
             ('deep', 1218),
             ('crater', 3683),
             ('near', 12938),
             ("city's", 726),
             ('airport', 5586),
             ('associated', 4582),
  

In [28]:
tokenizer.word_docs

defaultdict(int,
            {'measured': 202,
             'gunfire': 1520,
             'force': 3979,
             'national': 5819,
             'skimmed': 36,
             'night': 5767,
             'deep': 1078,
             'saw': 3833,
             'weekend': 3043,
             'humberto': 440,
             'international': 5105,
             'strauch': 422,
             'studies': 1092,
             'be': 35181,
             'a': 48457,
             'ice': 363,
             'sky': 899,
             'base': 1128,
             'expansive': 432,
             'close': 4250,
             'only': 11694,
             'parliament': 2257,
             'reports': 15417,
             '39': 1045,
             'depth': 335,
             'what': 17698,
             'that': 45001,
             'state': 15540,
             'disintegrated': 485,
             'come': 7282,
             'wilfried': 394,
             'saballos': 346,
             'astronomy': 252,
             'journalists': 195

In [29]:
tokenizer.document_count

49972

#### fit_on_texts() gives the following attributes in the output as given [here](https://faroit.github.io/keras-docs/1.2.2/preprocessing/text/).

* **word_counts:** dictionary mapping words (str) to the number of times they appeared on during fit. Only set after fit_on_texts was called.

* **word_docs:** dictionary mapping words (str) to the number of documents/texts they appeared on during fit. Only set after fit_on_texts was called.

* **word_index:** dictionary mapping words (str) to their rank/index (int). Only set after fit_on_texts was called.

* **document_count:** int. Number of documents (texts/sequences) the tokenizer was trained on. Only set after fit_on_texts or fit_on_sequences was called.



### Now, tokenize the sentences using nltk sent_tokenize() and encode the senteces with the ids we got form the above `t.word_index`

Initialise 2 lists with names `texts` and `articles`.

```
texts = [] to store text of article as it is.

articles = [] split the above text into a list of sentences.
```

In [0]:
texts = train_df['articleBody']

In [31]:
texts[0]

'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

## Check 2:

first element of texts and articles should be as given below. 

In [0]:
from nltk import sent_tokenize
articles = []

for txt in texts:
  new_sen = sent_tokenize(txt)
  articles.append(new_sen)

In [43]:
articles[0]

["A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday.",
 "Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports.",
 'Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth."',
 'House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports.',
 'Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened.',
 'The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee.',
 'He said it is still not clear if the meteorite disintegrated or was bu

# Now iterate through each article and each sentence to encode the words into ids using t.word_index  [5 marks] 

Here, to get words from sentence you can use `text_to_word_sequence` from keras preprocessing text.

1. Import text_to_word_sequence

2. Initialize a variable of shape (no.of articles, MAX_SENTS, MAX_SENT_LENGTH) with name `data` with zeros first (you can use numpy [np.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html) to initialize with all zeros)and then update it while iterating through the words and sentences in each article.

In [0]:
from keras_preprocessing.text import text_to_word_sequence

In [0]:
data = np.zeros((len(texts),MAX_SENTS, MAX_SENT_LENGTH),dtype='int')

In [56]:
data

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [0]:
exc = []
for i, sentences in enumerate(articles):
  for j, sent in enumerate(sentences):
    if j < MAX_SENTS:
      wordTokens = text_to_word_sequence(sent)
      k = 0
      for _, word in enumerate(wordTokens):
        try:
          if k < MAX_SENT_LENGTH and tokenizer.word_index[word] < MAX_NB_WORDS:
            data[i, j, k] = tokenizer.word_index[word]
            k = k + 1
        except:
          exc.append(word)
          pass

### Check 3:

Accessing first element in data should give something like given below.

In [128]:
data.shape

(49972, 20, 20)

In [129]:
data[0, :, :]

array([[    3,   485,   433,  7204,    81,     3,  3732,   331,     5,
         3888,   350,     4,  1432,  2956,     1,    89,    12,   464,
         6181,  6181],
       [  757,    95,  1045,     3,  2675,  1750,     7,   188,     3,
         1217,  1074,  2026,   698,   158,     1,  3029,   449,     1,
          555,   243],
       [   89,  1065,  4111,  2345,    12,     3,  1092,  3300,    19,
            1,    89,     2,  1791,     1,   529,  2005,    15,     9,
            3,  3107],
       [  186,  3639,   971,   202,  2553,    43,  6770,  1719,  1250,
            5, 13306, 17921,     1,   776,    31,   738,  3986,    67,
           85,  2153],
       [ 2345,    12,  1584,    38,  1094,   351,   777,     2,   367,
          260,  1775,     5,  4447,    70,   494,  1854,  1854,  1854,
         1854,  1854],
       [    1,   698,   188,    19,     1,   433,    32,     3,  7411,
            4,  2256,  1250,     6,     3,  5266,     4,  1217,  1250,
           12,  3359],
       [  

In [0]:
###data[0, :, :]

array([[    3,   487,   474,  7113,    79,     3,  3687,   325,     5,
         4200,   361,     4,  1525,  2913,     1,    89,    12,   451,
            0,     0],
       [  743,    96,  1044,     3,  2814,  1759,     7,   186,     3,
         1219,  1070,  1987,   736,   154,     1,  2990,   458,     1,
          543,   232],
       [   89,  1052,  4057,  2314,    12,     3,  1073,  3248,    19,
            1,    89,     2,  1751,     1,   518,  1980,    15,     9,
            3,  2879],
       [  182,  3691,   976,   196,  2515,    42,  6688,  1691,  1227,
            5, 13011, 17379,     1,   762,    30,   722,  3931,    66,
           87,     0],
       [ 2314,    12,  1882,    38,  1076,   346,   793,     2,   356,
          261,  1782,     5,  4396,    67,   486,     0,     0,     0,
            0,     0],
       [    1,   736,   186,    19,     1,   474,    32,     3,  7307,
            4,  2122,  1227,     6,     3,  5195,     4,  1219,  1227,
           12,  3308],
       [  

# Repeat the same process for the `Headings` as well. Use variables with names `texts_heading` and `articles_heading` accordingly. [5 marks] 

In [0]:
heading = train_df['Headline']

In [0]:
headlines = []

for txt in heading:
  new_sen = sent_tokenize(txt)
  headlines.append(new_sen)

In [50]:
headlines[0]

['Soldier shot, Parliament locked down after gunfire erupts at war memorial']

In [0]:
data_heading = np.zeros((len(texts),MAX_SENTS, MAX_SENT_LENGTH),dtype='int')
exc1 = []
for i, sentences in enumerate(headlines):
  for j, sent in enumerate(sentences):
    if j < MAX_SENTS:
      wordTokens = text_to_word_sequence(sent)
      k = 0
      for _, word in enumerate(wordTokens):
        try:
          if k < MAX_SENT_LENGTH and tokenizer.word_index[word] < MAX_NB_WORDS:
            data_heading[i, j, k] = tokenizer.word_index[word]
            k = k + 1
        except:
          exc1.append(word)
          pass

In [68]:
data_heading[0, :, :]

array([[  733,   209,   343,  7112,   192,    34,  1334, 11487,    21,
          233,   683,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [  

### Now the features are ready, lets make the labels ready for the model to process.

### Convert labels into one-hot vectors

You can use [get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) in pandas to create one-hot vectors.

In [0]:
labels = pd.get_dummies(train_df['Stance'])

### Check 4:

The shape of data and labels shoould match the given below numbers.

In [64]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (49972, 20, 20)
Shape of label tensor: (49972, 4)


### Shuffle the data

In [0]:
## get numbers upto no.of articles
indices = np.arange(data.shape[0])
## shuffle the numbers
np.random.shuffle(indices)

The variable name used below has no hint in the above lines. While executing the below lines we again have to go back and change the variable name. This should be improved in assignmet sheet

In [0]:
labels = labels.values

In [0]:
## shuffle the data
data = data[indices]
data_heading = data_heading[indices]
## shuffle the labels according to data
labels = labels[indices]

### Split into train and validation sets. Split the train set 80:20 ratio to get the train and validation sets.


Use the variable names as given below:

x_train, x_val - for body of articles.

x-heading_train, x_heading_val - for heading of articles.

y_train - for training labels.

y_val - for validation labels.



In [78]:
data.shape[0]

49972

In [80]:
49972*.2

9994.400000000001

In [0]:
x_train = data[:-9994]
x_val = data[-9994:]
y_train = labels[:-9994]
y_val = labels[-9994:]
x_heading_train = data_heading[:-9994]
x_heading_val = data_heading[-9994:]

### Check 5:

The shape of x_train, x_val, y_train and y_val should match the below numbers.

In [82]:
print(x_train.shape)
print(y_train.shape)

print(x_val.shape)
print(y_val.shape)

(39978, 20, 20)
(39978, 4)
(9994, 20, 20)
(9994, 4)


### Create embedding matrix with the glove embeddings


Run the below code to create embedding_matrix which has all the words and their glove embedding if present in glove word list.

In [100]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('/content/drive/My Drive/DL Project/Sequence NLP/glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
vocab_size = len(tokenizer.word_index)
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))


for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


# Try the sequential model approach and report the accuracy score. [10 marks]  

### Import layers from Keras to build the model

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Embedding, LSTM, Bidirectional
from keras import Input, Model

### Model

In [102]:
vocab_size

33642

In [115]:
model = Sequential()
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix]))
model.add(Bidirectional (LSTM (100, return_sequences = True , dropout = 0.2, recurrent_dropout = 0.2), merge_mode='concat'))
model.add(Dense(64,activation='relu'))
model.add(Dense(3,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, None, 100)         3364200   
_________________________________________________________________
bidirectional_8 (Bidirection (None, None, 200)         160800    
_________________________________________________________________
dense_19 (Dense)             (None, None, 64)          12864     
_________________________________________________________________
dense_20 (Dense)             (None, None, 3)           195       
Total params: 3,538,059
Trainable params: 3,538,059
Non-trainable params: 0
_________________________________________________________________


### Compile and fit the model

# I am not sure why I am getting error. Somewhere the code snippet provided is not correct


In [116]:
model.fit(x_train, y_train, validation_data=(x_val, y_val))

ValueError: ignored

## Build the same model with attention layers included for better performance (Optional)

## Fit the model and report the accuracy score for the model with attention layer (Optional)